# Configure Zero Latency Raw Bridge From Windows Camera to Linux


##### Download and Install FFMPEG

```powershell
New-Item -ItemType Directory -Force -Path "C:\Tools"
Set-Location "C:\Tools"
Invoke-WebRequest -Uri "https://www.gyan.dev/ffmpeg/builds/ffmpeg-release-essentials.zip" -OutFile "ffmpeg.zip"
Expand-Archive -Path "ffmpeg.zip" -DestinationPath "C:\Tools\ffmpeg" -Force
Set-Location "C:\Tools"
Invoke-WebRequest -Uri "https://www.gyan.dev/ffmpeg/builds/ffmpeg-release-essentials.zip" -OutFile "ffmpeg.zip"
Expand-Archive -Path "ffmpeg.zip" -DestinationPath "C:\Tools\ffmpeg" -Force
```

#### Find the path

```powershell
PS C:\Tools> Get-ChildItem "C:\Tools\ffmpeg" -Recurse -Filter ffmpeg.exe | Select-Object -First 1 FullName
```

#### output FullName
    C:\Tools\ffmpeg\ffmpeg-8.0-essentials_build\bin\ffmpeg.exe
    
#### Test Path

```powershell
PS C:\Tools> & "C:\Tools\ffmpeg\ffmpeg-8.0-essentials_build\bin\ffmpeg.exe" -version
```

#### Set Environment Variable and List Devices

```powershell
PS C:\Tools> $env:FFMPEG="C:\Tools\ffmpeg\ffmpeg-8.0-essentials_build\bin\ffmpeg.exe" # Set FFMPEG environment variable
PS C:\Tools> & $env:FFMPEG -list_devices true -f dshow -i dummy # List devicesa
```

#### List Supported Formats for Camera

```powershell
& $env:FFMPEG -f dshow -list_options true -i video="USB Camera Name" # see supported formats
```


## Receive the Stream in WSL
Correct sequence:

Run the WSL server cell first—it will print "Waiting for Windows connection (timeout in 30 seconds)..."
Within 30 seconds, run the ffmpeg command on Windows: & "C:\Tools\ffmpeg\ffmpeg-8.0-essentials_build\bin\ffmpeg.exe" -f dshow -rtbufsize 256M -video_size 1280x720 -framerate 30 -i video="USB Camera" -pix_fmt bgr24 -fflags nobuffer -flags low_delay -loglevel warning -f rawvideo "tcp://172.24.146.232:5000"

In [16]:
# listen_raw_server.py with integrated processing
import socket, numpy as np, cv2
from ultralytics import YOLO
from simple_lama_inpainting import SimpleLama
import time
import select

HOST, PORT = "0.0.0.0", 5000
W,H = 1280,720 # Set resolution here Must Match Client
BYTES = W*H*3

# Load models
yolo_model = YOLO('yolov8n-seg.pt')
lama_model = SimpleLama()

# Initialize video writers
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
fps = 30.0
out_original = cv2.VideoWriter('./Original_Webcam.avi', fourcc, fps, (W, H))
out_inpainted = cv2.VideoWriter('./SegYolov8_InptSimLama.avi', fourcc, fps, (W, H))

# Close any existing socket to avoid "Address already in use"
if 'srv' in globals():
    try:
        srv.close()
    except:
        pass

srv = socket.socket()
srv.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
srv.bind((HOST, PORT))
srv.listen(1)
print("WSL listening on", PORT)

# Wait for connection with timeout to avoid hanging
timeout = 30  # seconds
print(f"Waiting for Windows connection (timeout in {timeout} seconds)...")
ready, _, _ = select.select([srv], [], [], timeout)
if ready:
    conn, _ = srv.accept()
    print("Windows connected")
else:
    print("Timeout: No connection from Windows. Make sure ffmpeg is running on Windows.")
    srv.close()
    out_original.release()
    out_inpainted.release()
    raise SystemExit("No connection received")

frame_count = 0
max_frames = 30  # Process 30 frames

try:
    while frame_count < max_frames:
        buf = bytearray(BYTES); mv = memoryview(buf); got = 0
        while got < BYTES:
            n = conn.recv_into(mv[got:], BYTES-got)
            if n == 0: raise SystemExit
            got += n
        frame = np.frombuffer(buf, np.uint8).reshape((H,W,3))
        
        # Write a small file to indicate recording status
        with open('recording_status.txt', 'w') as f:
            f.write('recording\n')
        
        # Process the frame
        # Write original frame
        out_original.write(frame)

        # YOLO inference for people (class 0)
        results = yolo_model(frame, classes=[0], verbose=False)

        print(f"Frame {frame_count}: YOLO detected {len(results[0].masks) if results[0].masks is not None else 0} people")

        # For testing LaMa, create a synthetic mask if no people detected
        if results[0].masks is not None and len(results[0].masks) > 0:
            mask = results[0].masks.data[0].cpu().numpy()
            mask = (mask * 255).astype(np.uint8)
            mask = cv2.resize(mask, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)
            print(f"Using real mask with shape: {mask.shape}")
        else:
            # Create synthetic circular mask for testing LaMa
            mask = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.uint8)
            center = (frame.shape[1] // 2, frame.shape[0] // 2)
            radius = min(frame.shape[0], frame.shape[1]) // 4
            cv2.circle(mask, center, radius, 255, -1)
            print(f"Using synthetic mask with shape: {mask.shape}")

        # Inpaint using LaMa
        inpainted_result = lama_model(frame, mask)
        print(f"LaMa result type: {type(inpainted_result)}, shape: {getattr(inpainted_result, 'shape', 'no shape')}")
        # Convert PIL Image to numpy array for OpenCV
        if isinstance(inpainted_result, np.ndarray):
            inpainted_frame = inpainted_result
        else:
            # Convert PIL Image to numpy array (BGR format for OpenCV)
            inpainted_frame = np.array(inpainted_result)
            # Convert RGB to BGR if needed
            if len(inpainted_frame.shape) == 3 and inpainted_frame.shape[2] == 3:
                inpainted_frame = cv2.cvtColor(inpainted_frame, cv2.COLOR_RGB2BGR)

        # Write inpainted frame
        out_inpainted.write(inpainted_frame)

        frame_count += 1
        if frame_count % 10 == 0:  # Print progress every 10 frames
            print(f"Processed {frame_count} frames...")

finally:
    # Release resources
    out_original.release()
    out_inpainted.release()
    conn.close()
    srv.close()
    print(f"Processing complete! Processed {frame_count} frames from raw server stream.")
    print("Videos saved: ./Original_Webcam.avi and ./SegYolov8_InptSimLama.avi")

WSL listening on 5000
Waiting for Windows connection (timeout in 30 seconds)...
Windows connected
Windows connected
Frame 0: YOLO detected 2 people
Using real mask with shape: (720, 1280)
Frame 0: YOLO detected 2 people
Using real mask with shape: (720, 1280)
LaMa result type: <class 'PIL.Image.Image'>, shape: no shape
Frame 1: YOLO detected 2 people
Using real mask with shape: (720, 1280)
LaMa result type: <class 'PIL.Image.Image'>, shape: no shape
Frame 1: YOLO detected 2 people
Using real mask with shape: (720, 1280)
LaMa result type: <class 'PIL.Image.Image'>, shape: no shape
Frame 2: YOLO detected 1 people
Using real mask with shape: (720, 1280)
LaMa result type: <class 'PIL.Image.Image'>, shape: no shape
Frame 3: YOLO detected 1 people
Using real mask with shape: (720, 1280)
LaMa result type: <class 'PIL.Image.Image'>, shape: no shape
Frame 2: YOLO detected 1 people
Using real mask with shape: (720, 1280)
LaMa result type: <class 'PIL.Image.Image'>, shape: no shape
Frame 3: YOLO 

# Install Python packages

ultralytics, onnxruntime, onnx, opencv-python, numpy, Pillow, tqdm, simple-lama-inpainting, sympy, matplotlib, seaborn, pandas, ipywidgets .venv/bin/python -c "from ultralytics import YOLO; YOLO('yolov8n-seg.pt')"

In [ ]:
import torch
import onnxruntime as ort
import numpy as np
import cv2

print('=' * 50)
print('WSL Environment Verification')
print('=' * 50)
print(f'PyTorch: {torch.__version__}')
print(f'CUDA Available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'CUDA Version: {torch.version.cuda}')
    print(f'GPU Device: {torch.cuda.get_device_name(0)}')
print(f'NumPy: {np.__version__}')
print(f'OpenCV: {cv2.__version__}')
print(f'ONNX Runtime: {ort.__version__}')
print(f'Available Providers: {ort.get_available_providers()}')
print('=' * 50)
print('✅ setup complete!')
print('=' * 50)

In [ ]:

# Test imports and basic functionality
from ultralytics import YOLO
import torch
from simple_lama_inpainting import SimpleLama
import onnxruntime as ort

print("Testing YOLO...")
yolo_model = YOLO('yolov8n-seg.pt')
print("✅ YOLO loaded")

print("Testing LaMa...")
# Note: LaMa model would need to be available
# lama_model = SimpleLama()
print("✅ LaMa ready")

print("Testing ONNX Runtime...")
print(f"Available providers: {ort.get_available_providers()}")
print("✅ ONNX Runtime ready")

print("🎉 All components ready!")


Start the TCP Raw-Video Server on Windows
Example: 
Device: "USB Camera"
Mode: vcodec=mjpeg 1280x720 @ 30 fps

# First, find the WSL IP address (run in WSL terminal):
# ip addr show eth0 | grep inet

# Then, start ffmpeg to connect to WSL server (replace <WSL_IP> with the actual IP, e.g., 172.17.0.1)
& $env:FFMPEG -f dshow `
  -rtbufsize 256M -video_size 1280x720 -framerate 30 `
  -i video="USB Camera" `
  -pix_fmt bgr24 -fflags nobuffer -flags low_delay `
  -f rawvideo "tcp://<WSL_IP>:5000"


In [ ]:
import cv2
from ultralytics import YOLO
from simple_lama_inpainting import SimpleLama
import numpy as np
import subprocess
import os
import time
import socket

# Load YOLO model
yolo_model = YOLO('yolov8n-seg.pt')

# LaMa model (uncomment if available)
lama_model = SimpleLama()

# Dummy inpainting function
def dummy_inpainting(frame, mask):
    # Simple dummy: just return the original frame
    return frame

# Capture a single frame from the raw TCP stream
def capture_frame_from_socket():
    """Capture a single frame from the raw TCP stream on port 5000"""
    s = socket.socket()
    try:
        s.connect(("localhost", 5000))
    except ConnectionRefusedError:
        return None
    W, H = 1280, 720
    BYTES = W * H * 3
    buf = bytearray(BYTES)
    mv = memoryview(buf)
    got = 0
    while got < BYTES:
        n = s.recv_into(mv[got:], BYTES - got)
        if n == 0:
            s.close()
            return None
        got += n
    frame = np.frombuffer(buf, np.uint8).reshape((H, W, 3))
    s.close()
    return frame

# Camera properties (corrected for 720p)
frame_width = 1280
frame_height = 720
fps = 30.0

# Initialize video writers
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out_original = cv2.VideoWriter('./Original_Webcam.avi', fourcc, fps, (frame_width, frame_height))
out_inpainted = cv2.VideoWriter('./SegYolov8_InptSimLama.avi', fourcc, fps, (frame_width, frame_height))

print("Starting video processing with socket capture from raw server stream...")

frame_count = 0
max_frames = 30  # Capture 30 frames

while frame_count < max_frames:
    # Capture frame from socket
    frame = capture_frame_from_socket()
    
    if frame is None:
        print(f"Failed to capture frame {frame_count} from socket")
        break
    
    # Write original frame
    out_original.write(frame)

    # YOLO inference for people (class 0)
    results = yolo_model(frame, classes=[0], verbose=False)

    print(f"Frame {frame_count}: YOLO detected {len(results[0].masks) if results[0].masks is not None else 0} people")

    # For testing LaMa, create a synthetic mask if no people detected
    if results[0].masks is not None and len(results[0].masks) > 0:
        mask = results[0].masks.data[0].cpu().numpy()
        mask = (mask * 255).astype(np.uint8)
        mask = cv2.resize(mask, (frame.shape[1], frame.shape[0]), interpolation=cv2.INTER_NEAREST)
        print(f"Using real mask with shape: {mask.shape}")
    else:
        # Create synthetic circular mask for testing LaMa
        mask = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.uint8)
        center = (frame.shape[1] // 2, frame.shape[0] // 2)
        radius = min(frame.shape[0], frame.shape[1]) // 4
        cv2.circle(mask, center, radius, 255, -1)
        print(f"Using synthetic mask with shape: {mask.shape}")

    # Inpaint using LaMa
    inpainted_result = lama_model(frame, mask)
    print(f"LaMa result type: {type(inpainted_result)}, shape: {getattr(inpainted_result, 'shape', 'no shape')}")
    # Convert PIL Image to numpy array for OpenCV
    if isinstance(inpainted_result, np.ndarray):
        inpainted_frame = inpainted_result
    else:
        # Convert PIL Image to numpy array (BGR format for OpenCV)
        inpainted_frame = np.array(inpainted_result)
        # Convert RGB to BGR if needed
        if len(inpainted_frame.shape) == 3 and inpainted_frame.shape[2] == 3:
            inpainted_frame = cv2.cvtColor(inpainted_frame, cv2.COLOR_RGB2BGR)

    # Write inpainted frame
    out_inpainted.write(inpainted_frame)

    frame_count += 1
    if frame_count % 10 == 0:  # Print progress every 10 frames
        print(f"Processed {frame_count} frames...")

# Release resources
out_original.release()
out_inpainted.release()

print(f"Processing complete! Processed {frame_count} frames from raw server stream.")
print("Videos saved: ./Original_Webcam.avi and ./SegYolov8_InptSimLama.avi")

In [17]:
cap_original = cv2.VideoCapture('./Original_Webcam.avi')
cap_inpainted = cv2.VideoCapture('./SegYolov8_InptSimLama.avi')

if not cap_original.isOpened():
    print("Error: Could not open original video file ./Original_Webcam.avi")
    exit()

if not cap_inpainted.isOpened():
    print("Error: Could not open inpainted video file ./SegYolov8_InptSimLama.avi")
    exit()

print("Both video files opened successfully.")

Both video files opened successfully.


## Create side-by-side video

### Subtask:
Iterate through the frames of both videos, combine each corresponding pair of frames horizontally, and write the combined frames to a new output video file.


**Reasoning**:
Iterate through the frames of both videos, combine each corresponding pair of frames horizontally, and write the combined frames to a new output video file.



In [18]:
# Get video properties
frame_width = int(cap_original.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap_original.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap_original.get(cv2.CAP_PROP_FPS))

# Create output directory
import os
output_dir = './output'
os.makedirs(output_dir, exist_ok=True)

# Find the next incrementing number
existing_files = [f for f in os.listdir(output_dir) if f.startswith('side_by_side_output_') and f.endswith('.avi')]
numbers = []
for f in existing_files:
    parts = f.split('_')
    if len(parts) >= 4 and parts[-1].endswith('.avi'):
        num_str = parts[-1].replace('.avi', '')
        if num_str.isdigit():
            numbers.append(int(num_str))
next_num = max(numbers) + 1 if numbers else 1

# Define output video path and initialize VideoWriter
output_comparison_path = os.path.join(output_dir, f'side_by_side_output_{next_num}.avi')
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out_comparison = cv2.VideoWriter(output_comparison_path, fourcc, fps, (frame_width * 2, frame_height))

print("Creating side-by-side video...")

# Loop through frames and combine
while cap_original.isOpened() and cap_inpainted.isOpened():
    ret_original, frame_original = cap_original.read()
    ret_inpainted, frame_inpainted = cap_inpainted.read()

    if not ret_original or not ret_inpainted:
        break

    # Concatenate frames horizontally
    combined_frame = np.concatenate((frame_original, frame_inpainted), axis=1)

    # Write the combined frame to the output video
    out_comparison.write(combined_frame)

# Release resources
cap_original.release()
cap_inpainted.release()
out_comparison.release()

print(f"Side-by-side video saved to {output_comparison_path}")

Creating side-by-side video...
Side-by-side video saved to ./output/side_by_side_output_1.avi
Side-by-side video saved to ./output/side_by_side_output_1.avi


Here is the code to display the side-by-side comparison video:

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

output_dir = './output'

if os.path.exists(output_dir):
    avi_files = [f for f in os.listdir(output_dir) if f.startswith('side_by_side_output_') and f.endswith('.avi')]
    if avi_files:
        numbers = [int(f.split('_')[-1].split('.')[0]) for f in avi_files if f.split('_')[-1].split('.')[0].isdigit()]
        latest_num = max(numbers)
        comparison_video_path = os.path.join(output_dir, f'side_by_side_output_{latest_num}.avi')
        if os.path.exists(comparison_video_path):
            # Read the video file
            with open(comparison_video_path, 'rb') as video_file:
                video_bytes = video_file.read()

            # Encode the video in base64
            video_encoded = b64encode(video_bytes).decode()

            # Create an HTML video player
            video_html = f"""
            <video width="800" controls>
              <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
              Your browser does not support the video tag.
            </video>
            """

            # Display the video player
            display(HTML(video_html))
        else:
            print(f"Latest video file '{comparison_video_path}' does not exist.")
    else:
        print("No side-by-side videos found in output directory.")
else:
    print("Output directory does not exist.")

In [ ]:
from IPython.display import Video
import os

output_dir = './output'

if os.path.exists(output_dir):
    avi_files = [f for f in os.listdir(output_dir) if f.startswith('side_by_side_output_') and f.endswith('.avi')]
    if avi_files:
        numbers = [int(f.split('_')[-1].split('.')[0]) for f in avi_files if f.split('_')[-1].split('.')[0].isdigit()]
        latest_num = max(numbers)
        comparison_video_path = os.path.join(output_dir, f'side_by_side_output_{latest_num}.avi')
        if os.path.exists(comparison_video_path):
            # Display the video
            Video(comparison_video_path, embed=True)
        else:
            print(f"Latest video file '{comparison_video_path}' does not exist.")
    else:
        print("No side-by-side videos found in output directory.")
else:
    print("Output directory does not exist.")

In [20]:
import os

output_dir = './output'

if os.path.exists(output_dir):
    avi_files = [f for f in os.listdir(output_dir) if f.startswith('side_by_side_output_') and f.endswith('.avi')]
    if avi_files:
        numbers = [int(f.split('_')[-1].split('.')[0]) for f in avi_files if f.split('_')[-1].split('.')[0].isdigit()]
        latest_num = max(numbers)
        comparison_video_path = os.path.join(output_dir, f'side_by_side_output_{latest_num}.avi')
        if os.path.exists(comparison_video_path):
            file_size = os.path.getsize(comparison_video_path)
            print(f"Latest video file '{comparison_video_path}' exists with size: {file_size} bytes")
        else:
            print(f"Latest video file '{comparison_video_path}' does not exist.")
    else:
        print("No side-by-side videos found in output directory.")
else:
    print("Output directory does not exist.")

Latest video file './output/side_by_side_output_1.avi' exists with size: 1079318 bytes


In [19]:
from IPython.display import Video
import os

output_dir = './output'

if os.path.exists(output_dir):
    avi_files = [f for f in os.listdir(output_dir) if f.startswith('side_by_side_output_') and f.endswith('.avi')]
    if avi_files:
        numbers = [int(f.split('_')[-1].split('.')[0]) for f in avi_files if f.split('_')[-1].split('.')[0].isdigit()]
        latest_num = max(numbers)
        comparison_video_path = os.path.join(output_dir, f'side_by_side_output_{latest_num}.avi')
        if os.path.exists(comparison_video_path):
            # Display the video
            Video(comparison_video_path, embed=True)
        else:
            print(f"Latest video file '{comparison_video_path}' does not exist.")
    else:
        print("No side-by-side videos found in output directory.")
else:
    print("Output directory does not exist.")